In [2]:
import tensorflow as tf
from tensorflow.keras import models , layers
import numpy as np

In [3]:
IMAGE_SIZE=256
BATCH_SIZE=8
CHANNELS=3

In [4]:
dataset =tf.keras.preprocessing.image_dataset_from_directory(
    "Apple",
    shuffle=True,
    image_size = (IMAGE_SIZE,IMAGE_SIZE),
    batch_size = BATCH_SIZE
)

Found 9714 files belonging to 4 classes.


In [5]:
class_names = dataset.class_names
class_names

['Apple___Apple_scab',
 'Apple___Black_rot',
 'Apple___Cedar_apple_rust',
 'Apple___healthy']

In [6]:
len(dataset)

1215

In [7]:
def get_dataset_partitions_tf(ds,train_split=0.8, val_split=0.1, test_split=0.1, shuffle=True, shuffle_size=1000):

  ds_size =len(ds)

  if shuffle:
    ds=ds.shuffle(shuffle_size)

  train_size = int(train_split*ds_size)
  val_size = int(val_split*ds_size)

  train_ds = ds.take(train_size)

  val_ds=ds.skip(train_size).take(val_size)
  test_ds=ds.skip(train_size).skip(val_size)


  return train_ds ,val_ds, test_ds

In [8]:
train_ds, val_ds, test_ds = get_dataset_partitions_tf(dataset)


In [9]:
len(train_ds)

972

In [10]:
len(val_ds)

121

In [11]:
len(test_ds)

122

In [12]:
train_ds=train_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
val_ds=val_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)
test_ds=test_ds.cache().shuffle(1000).prefetch(buffer_size=tf.data.AUTOTUNE)

In [13]:
resize_and_rescale = tf.keras.Sequential([
    layers.Resizing(IMAGE_SIZE, IMAGE_SIZE),
    layers.Rescaling(1.0/255)

])

In [14]:
data_augmentation = tf.keras.Sequential([
     layers.RandomFlip("horizontal_and_vertical"),
     layers.RandomRotation(0.2),
])

In [15]:
input_shape=(BATCH_SIZE, IMAGE_SIZE, IMAGE_SIZE, CHANNELS)
n_classes=4
model1=models.Sequential([
    resize_and_rescale,
    data_augmentation,

    layers.Conv2D(32,(3,3),activation='relu',input_shape=input_shape),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),


    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),


    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),

    layers.Conv2D(64,(3,3),activation='relu'),
    layers.MaxPooling2D(2,2),


    layers.Flatten(),

    layers.Dense(64,activation='relu'),
    layers.Dense(n_classes,activation='softmax')
])
model1.build(input_shape=input_shape)

c:\Users\rekha\OneDrive\Desktop\Plant Disease Detection\venv\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [16]:
model1.summary

<bound method Model.summary of <Sequential name=sequential_2, built=True>>

In [17]:
model1.compile(
    optimizer='adam',
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=False),
    metrics=['accuracy']
)

In [18]:
from tensorflow.keras.callbacks import ModelCheckpoint

In [19]:
checkpoint = ModelCheckpoint('./apple_model.keras',
    verbose=1,
    monitor='val_accuracy',
    save_best_only=True,
    mode='auto'
)

In [20]:
EPOCHS=15
history=model1.fit(
    train_ds,
    epochs=EPOCHS,
    batch_size=BATCH_SIZE,
    verbose=1,
    validation_data=val_ds,
    callbacks=[checkpoint]
)

Epoch 1/15
972/972 ━━━━━━━━━━━━━━━━━━━━ 0s 507ms/step - accuracy: 0.5125 - loss: 1.0283
Epoch 1: val_accuracy improved from -inf to 0.85537, saving model to ./apple_model.keras
972/972 ━━━━━━━━━━━━━━━━━━━━ 577s 544ms/step - accuracy: 0.5126 - loss: 1.0280 - val_accuracy: 0.8554 - val_loss: 0.3581
Epoch 2/15
972/972 ━━━━━━━━━━━━━━━━━━━━ 0s 498ms/step - accuracy: 0.8847 - loss: 0.2970
Epoch 2: val_accuracy did not improve from 0.85537
972/972 ━━━━━━━━━━━━━━━━━━━━ 501s 513ms/step - accuracy: 0.8847 - loss: 0.2970 - val_accuracy: 0.8502 - val_loss: 0.4058
Epoch 3/15
972/972 ━━━━━━━━━━━━━━━━━━━━ 0s 499ms/step - accuracy: 0.9343 - loss: 0.1890
Epoch 3: val_accuracy improved from 0.85537 to 0.93905, saving model to ./apple_model.keras
972/972 ━━━━━━━━━━━━━━━━━━━━ 500s 514ms/step - accuracy: 0.9343 - loss: 0.1889 - val_accuracy: 0.9390 - val_loss: 0.1576
Epoch 4/15
972/972 ━━━━━━━━━━━━━━━━━━━━ 0s 523ms/step - accuracy: 0.9631 - loss: 0.1118
Epoch 4: val_accuracy did not improve from 0.93905
97

In [21]:
scores=model1.evaluate(test_ds)

122/122 ━━━━━━━━━━━━━━━━━━━━ 47s 102ms/step - accuracy: 0.9602 - loss: 0.0928


In [22]:
history.params

{'verbose': 1, 'epochs': 15, 'steps': 972}

In [23]:
history.history.keys()

dict_keys(['accuracy', 'loss', 'val_accuracy', 'val_loss'])

In [24]:
history.history['accuracy']

[0.6904761791229248,
 0.900772213935852,
 0.9437580704689026,
 0.9651222825050354,
 0.9630630612373352,
 0.970141589641571,
 0.976962685585022,
 0.9676962494850159,
 0.9796653985977173,
 0.9821106791496277,
 0.9790219068527222,
 0.9876447916030884,
 0.9841699004173279,
 0.9849420785903931,
 0.9855855703353882]

In [25]:
acc=history.history['accuracy']
val_acc=history.history['val_accuracy']

loss=history.history['loss']
val_loss=history.history['val_loss']

In [26]:
val_acc

[0.85537189245224,
 0.8502066135406494,
 0.9390496015548706,
 0.9380165338516235,
 0.9772727489471436,
 0.8760330677032471,
 0.98037189245224,
 0.9493801593780518,
 0.9741735458374023,
 0.9865702390670776,
 0.9845041036605835,
 0.9597107172012329,
 0.961776852607727,
 0.9865702390670776,
 0.9545454382896423]

In [27]:
import numpy as np
for images_batch,label_batch in test_ds.take(1):
  first_image = images_batch[0].numpy().astype('uint8')
  first_label = label_batch[0].numpy()

  print('first image to predict:')

  #print("first image's actual label:",first_label)
  print("first image's actual label:",class_names[first_label])
  batch_prediction = model1.predict(images_batch)
  print(batch_prediction[0])
  print(np.argmax(batch_prediction[0]))
  print(class_names[np.argmax(batch_prediction[0])])




first image to predict:
first image's actual label: Apple___Cedar_apple_rust
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 423ms/step
[6.5581644e-06 2.6280975e-06 9.9999082e-01 1.5671032e-11]
2
Apple___Cedar_apple_rust


In [28]:
def predict(model,img):
  img_array = tf.keras.preprocessing.image.img_to_array(images[i].numpy())
  img_array = tf.expand_dims(img_array,0)# Create a batch

  predictions = model.predict(img_array)

  predicted_class = class_names[np.argmax(predictions[0])]
  confidence = round(100* (np.max(predictions[0])),2)
  return predicted_class, confidence

In [29]:

for images, labels in test_ds.take(1):
  for i in range(9):

    predicted_class, confidence=predict(model1,images[i].numpy())
    print(predicted_class)
    print(confidence)
    actual_class = class_names[labels[i]]
    print(actual_class)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 293ms/step
Apple___Black_rot
100.0
Apple___Black_rot
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 55ms/step
Apple___Black_rot
100.0
Apple___Black_rot
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 67ms/step
Apple___healthy
99.43
Apple___healthy
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 49ms/step
Apple___Apple_scab
99.84
Apple___Apple_scab
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 50ms/step
Apple___Cedar_apple_rust
100.0
Apple___Cedar_apple_rust
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step
Apple___Apple_scab
91.68
Apple___Apple_scab
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 56ms/step
Apple___Apple_scab
99.93
Apple___Apple_scab
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 54ms/step
Apple___Apple_scab
94.16
Apple___healthy


InvalidArgumentError: {{function_node __wrapped__StridedSlice_device_/job:localhost/replica:0/task:0/device:CPU:0}} slice index 8 of dimension 0 out of bounds. [Op:StridedSlice] name: strided_slice/